In [ ]:
# Install deps
# !python -m spacy download en_core_web_md
# !pip install torch torchvision
# !pip install transformers
# !pip3 install nltk emoji==0.6.0
# !pip install optuna

In [47]:
#dataset
import pandas as pd
from datasets import Dataset 
# utils
from sklearn.utils import shuffle
from sklearn.metrics import classification_report
#visualize
import matplotlib.pyplot as plt
%matplotlib inline

# modeling
import spacy
from transformers import AutoTokenizer, TFAutoModel
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import recall_score, precision_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import numpy as np

from transformers import BertTokenizer, BertForSequenceClassification
from transformers import Trainer, TrainingArguments
from datasets import load_metric
from transformers import AutoTokenizer, AutoModelForSequenceClassification



Reader functions:

In [33]:
# We define a function to read our already pre-processed data
def reader_df(topic):
  path_train = "cleaned_df/stance_" + topic + "_train_cleaned.csv"
  path_test = "cleaned_df/stance_" + topic + "_test_cleaned.csv"
  path_val = "cleaned_df/stance_" + topic + "_validation_cleaned.csv"
  df_train = pd.read_csv(path_train)
  df_val = pd.read_csv(path_val)
  df_test = pd.read_csv(path_test)

  X_train = df_train.loc[:, 'text'].values
  y_train = df_train.loc[:, 'label'].values

  X_test = df_test.loc[:, 'text'].values
  y_test = df_test.loc[:, 'label'].values

  X_val = df_val.loc[:, 'text'].values
  y_val = df_val.loc[:, 'label'].values

  return X_train, X_test, y_train, y_test, X_val, y_val

In [34]:
# During finetuning, we might want to use the specific tokenizer functions of BERT models, so we define a function to read the raw data.
def raw_reader_and_tokenize(topic):
    path_X_train = "https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/stance/" + topic + "/train_text.txt"
    path_X_test = "https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/stance/" + topic + "/test_text.txt"
    path_X_val = "https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/stance/" + topic + "/val_text.txt"
    path_y_train = "https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/stance/" + topic + "/train_labels.txt"
    path_y_test = "https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/stance/" + topic + "/test_labels.txt"
    path_y_val = "https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/stance/" + topic + "/val_labels.txt"

    X_train = pd.read_table(path_X_train, header=None)
    X_test = pd.read_table(path_X_test, header=None)
    X_val = pd.read_table(path_X_val, header=None)
    y_train = pd.read_table(path_y_train, header=None)
    y_test = pd.read_table(path_y_test, header=None)
    y_val = pd.read_table(path_y_val, header=None)

    X_train = X_train.rename(columns={0: "text"})
    X_test = X_test.rename(columns={0: "text"})
    y_train = y_train.rename(columns={0: "label"})
    y_test = y_test.rename(columns={0: "label"})
    X_val = X_val.rename(columns={0: "text"})
    y_val = y_val.rename(columns={0: "label"})

    # Convert the DataFrame to a Hugging Face Dataset
    train_dataset = Dataset.from_dict({"text": X_train["text"].values, "label": y_train["label"].values})
    val_dataset = Dataset.from_dict({"text": X_val["text"].values, "label": y_val["label"].values})
    test_dataset = Dataset.from_dict({"text": X_test["text"].values, "label": y_test["label"].values})

    # Tokenize the training and validation datasets
    train_dataset = train_dataset.map(tokenize, batched=True, batch_size=len(train_dataset))
    val_dataset = val_dataset.map(tokenize, batched=True, batch_size=len(val_dataset))
    test_dataset = test_dataset.map(tokenize, batched=True, batch_size=len(test_dataset))

    # Set the format of the datasets to PyTorch tensors
    train_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])
    val_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])
    test_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])


    return train_dataset, val_dataset, test_dataset
    
    

+ **DistilBERT** model

As a first test, we can calculate how accurate the predictions are using a DistilBert model. The code below uses `DistilBertTokenizerFast` on our already pre-processed datasets. 
We apply the `DistilBertForSequenceClassification` model, which is the most appropiate for our current task of multi-class classification.

We will implement this first model in the `stances-feminist` dataset, just to test it.

In [ ]:
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification
from sklearn.metrics import f1_score, precision_score, accuracy_score
import torch

X_train, X_test, y_train, y_test, X_val, y_val = reader_df("feminist")



# Load the tokenizer and model
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=3)

# Prepare the data
train_texts = X_train.tolist() 
train_labels = y_train.tolist()
test_texts = X_test.tolist()
test_labels = y_test.tolist()

# Tokenize the data
train_encodings = tokenizer(train_texts, return_tensors='pt', padding=True, truncation=True)
test_encodings = tokenizer(test_texts, return_tensors='pt', padding=True, truncation=True)

# Convert to torch Dataset
class TweetDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = TweetDataset(train_encodings, train_labels)
test_dataset = TweetDataset(test_encodings, test_labels)

# Train the model
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu') # useful to improve computing time, depending on our pc characteristics
model.to(device)
model.train()

In the code below we use the PyTorch library to create a DataLoader object for the train_dataset with a batch size of 16. This means that the data in train_dataset will be divided into batches of 16 samples each and fed into the model for training.
We also create an Adam optimizer object with a learning rate of 5e-5. The optimizer is used to update the model’s parameters during training to minimize the loss function.

In [ ]:

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16)
optim = torch.optim.Adam(model.parameters(), lr=5e-5)

for epoch in range(3):
    for batch in train_loader:
        optim.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs[0]
        loss.backward()
        optim.step()

# Evaluate the model on the test set
model.eval()
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=16)
predictions = []
true_labels = []
with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs[0]
        predictions.extend(logits.argmax(dim=-1).cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

# Calculate metrics
f1 = f1_score(true_labels, predictions, average='macro')
precision = precision_score(true_labels, predictions, average='macro')
accuracy = accuracy_score(true_labels, predictions)

print(f'F1 score: {f1}')
print(f'Precision: {precision}')
print(f'Accuracy: {accuracy}')

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.weight', 'pre_clas

F1 score: 0.5426585516619846
Precision: 0.6004088504088504
Accuracy: 0.5719298245614035


We have an F1 of 54 with this implementation. How can we improve this?

#### Fine-tuned BERTweet implementation

BERT is a pre-trained language model that has been shown to achieve state-of-the-art performance on a wide range of natural language processing tasks. However, BERT was pre-trained on a large corpus of general text, which may not be representative of the language used in tweets.

Tweets are known to have unique characteristics that can make them more challenging to classify compared to other types of text. For example, tweets are often shorter, contain a lot of noise (such as typos and slang), and can have complex grammatical structures that are not found in more formal writing.

Additionally, the use of hashtags, emojis, and other special characters in tweets can make it difficult for BERT to understand the context and sentiment of the tweet. Pre-processing and cleaning the tweets can help to mitigate some of these issues, but there is still a limit to the effectiveness of this approach.

To address these challenges, researchers have developed specialized versions of BERT for use with social media data. For example, BERTweet is a variant of BERT that has been trained specifically on tweets and has been shown to outperform generic BERT on tweet classification tasks.

+ **BERTweet**

First we define a `compute_metrics` function that calls precision, recall and F1, using a macro average.

In [41]:
precision = load_metric("precision")
recall = load_metric("recall")
f1 = load_metric("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {
        "precision": precision.compute(predictions=predictions, references=labels, average="macro"),
        "recall": recall.compute(predictions=predictions, references=labels, average="macro"),
        "f1": f1.compute(predictions=predictions, references=labels, average="macro"),
    }

We define the tokenizer from Bertweet, and implement the five models after that.

In [42]:
# Load the pre-trained BERTweet tokenizer
tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base", normalization=True)

# Define a function to tokenize the dataset
def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [43]:
# Load the pre-trained BERTweet model for sequence classification
model = AutoModelForSequenceClassification.from_pretrained("vinai/bertweet-base", num_labels=3)

Some weights of the model checkpoint at vinai/bertweet-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.bias', 'roberta.pooler.dense.weight', 'lm_head.bias', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/bertweet-base and are newly initialized: 

### Stance: climate

In this case, instead of our pre-processed datasets, we will import the raw data.
(As a note, we tried an implementation using our pre-processed datasets, but the scores were slightly worse in all cases. For this alternative implementation, see code that is commented out below.)

In [ ]:
train_dataset, val_dataset, test_dataset = raw_reader_and_tokenize("climate")



Map:   0%|          | 0/355 [00:00<?, ? examples/s]

Map:   0%|          | 0/40 [00:00<?, ? examples/s]

Map:   0%|          | 0/169 [00:00<?, ? examples/s]

In [ ]:
### ALTERNATIVE IMPLEMENTATION WITH OUR PRE-PROCESSED DATA, INSTEAD OF RAW DATA

# Convert the DataFrame to a Hugging Face Dataset
# X_train, X_test, y_train, y_test, X_val, y_val = reader_df("climate")

# train_dataset = Dataset.from_dict({"text": X_train, "label": y_train})
# val_dataset = Dataset.from_dict({"text": X_val, "label": y_val})
# test_dataset = Dataset.from_dict({"text": X_test, "label": y_test})

# # Tokenize the training and validation datasets
# train_dataset = train_dataset.map(tokenize, batched=True, batch_size=len(train_dataset))
# val_dataset = val_dataset.map(tokenize, batched=True, batch_size=len(val_dataset))

# # Set the format of the datasets to PyTorch tensors
# train_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])
# val_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])


In [ ]:
# Define the training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    save_strategy='epoch',
    learning_rate=1e-4,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=5,
    logging_dir='./logs',
    load_best_model_at_end=True,
    seed=1
)


# Create a Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

# Fine-tune the model on the training dataset
trainer.train()

loading configuration file config.json from cache at C:\Users\danid/.cache\huggingface\hub\models--vinai--bertweet-base\snapshots\118ab1d567653bec16bbb081eafb6f8942f72108\config.json
Model config RobertaConfig {
  "_name_or_path": "vinai/bertweet-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 130,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "tokenizer_class": "BertweetTokenizer",
  "transforme

  0%|          | 0/60 [00:00<?, ?it/s]

The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 40
  Batch size = 32


  0%|          | 0/2 [00:00<?, ?it/s]

c:\Users\danid\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Trainer is attempting to log a value of "{'precision': 0.5839727195225916}" of type <class 'dict'> for key "eval/precision" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'recall': 0.6115779645191409}" of type <class 'dict'> for key "eval/recall" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'f1': 0.5971479500891265}" of type <class 'dict'> for key "eval/f1" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Saving model checkp

...

c:\Users\danid\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Trainer is attempting to log a value of "{'precision': 0.6041666666666666}" of type <class 'dict'> for key "eval/precision" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'recall': 0.6274509803921569}" of type <class 'dict'> for key "eval/recall" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'f1': 0.6141414141414141}" of type <class 'dict'> for key "eval/f1" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Saving model checkp

{'eval_loss': 0.37791839241981506, 'eval_precision': {'precision': 0.6041666666666666}, 'eval_recall': {'recall': 0.6274509803921569}, 'eval_f1': {'f1': 0.6141414141414141}, 'eval_runtime': 1.7367, 'eval_samples_per_second': 23.032, 'eval_steps_per_second': 1.152, 'epoch': 5.0}


Model weights saved in ./results\checkpoint-60\pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from ./results\checkpoint-48 (score: 0.3590737581253052).


{'train_runtime': 333.8925, 'train_samples_per_second': 5.316, 'train_steps_per_second': 0.18, 'train_loss': 0.48289686838785806, 'epoch': 5.0}


TrainOutput(global_step=60, training_loss=0.48289686838785806, metrics={'train_runtime': 333.8925, 'train_samples_per_second': 5.316, 'train_steps_per_second': 0.18, 'train_loss': 0.48289686838785806, 'epoch': 5.0})

In [ ]:
# Evaluate the trained model in the test set

# trainer.compute_metrics=compute_metrics
# (if we want to use different metrics (e.g. weighted average), just update the compute_metrics function
#  and run the line above)
results = trainer.evaluate(test_dataset)




The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 169
  Batch size = 32


  0%|          | 0/6 [00:00<?, ?it/s]

c:\Users\danid\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Trainer is attempting to log a value of "{'precision': 0.5184569952011812}" of type <class 'dict'> for key "eval/precision" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'recall': 0.5770034843205575}" of type <class 'dict'> for key "eval/recall" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'f1': 0.5450520028833282}" of type <class 'dict'> for key "eval/f1" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


In [ ]:
print(results["eval_precision"])
print(results["eval_recall"])
print(results["eval_f1"])

{'precision': 0.5184569952011812}
{'recall': 0.5770034843205575}
{'f1': 0.5450520028833282}


To assess possible biases, we can check how well we are predicting each class:

In [ ]:
# Generate predictions for the test set
test_predictions = trainer.predict(test_dataset).predictions
test_predictions = np.argmax(test_predictions, axis=-1)

# Compute the classification report
report = classification_report(test_dataset['label'], test_predictions, output_dict=True)

# Print the classification report
for label in report:
    if label != 'accuracy':
        print(f"Class: {label}")
        print(f"Precision: {report[label]['precision']}")
        print(f"Recall: {report[label]['recall']}")
        print(f"F1-score: {report[label]['f1-score']}\n")

The following columns in the test set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 169
  Batch size = 32


  0%|          | 0/6 [00:00<?, ?it/s]

Class: 0
Precision: 0.6744186046511628
Recall: 0.8285714285714286
F1-score: 0.7435897435897435

Class: 1
Precision: 0.0
Recall: 0.0
F1-score: 0.0

Class: 2
Precision: 0.8809523809523809
Recall: 0.9024390243902439
F1-score: 0.8915662650602411

Class: macro avg
Precision: 0.5184569952011812
Recall: 0.5770034843205575
F1-score: 0.5450520028833282

Class: weighted avg
Precision: 0.7808390178694293
Recall: 0.8284023668639053
F1-score: 0.8028892995742644



c:\Users\danid\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\danid\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\danid\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\danid\anaconda3\lib\site-packages\sklearn\metrics\_classifi

To compare with the original benchmarking methodology, we take the average of F1 scores for against and favor clases:

In [ ]:
# Final evaluation
f1_against = report['1']['f1-score']
f1_favor = report['2']['f1-score']
tweeteval_result = (f1_against+f1_favor) / 2
print("Final F1 - Stance CLIMATE: " + str(tweeteval_result))

Final F1 - Stance CLIMATE: 0.44578313253012053


###  Stance: atheism

In [ ]:
# Import dataset
train_dataset, val_dataset, test_dataset = raw_reader_and_tokenize("atheism")


# Create a Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

# Fine-tune the model on the training dataset
trainer.train()

# Generate predictions for the test set
test_predictions = trainer.predict(test_dataset).predictions
test_predictions = np.argmax(test_predictions, axis=-1)

# Compute the classification report
report = classification_report(test_dataset['label'], test_predictions, output_dict=True)

# Print the classification report
for label in report:
    if label != 'accuracy':
        print(f"Class: {label}")
        print(f"Precision: {report[label]['precision']}")
        print(f"Recall: {report[label]['recall']}")
        print(f"F1-score: {report[label]['f1-score']}\n")

Map:   0%|          | 0/461 [00:00<?, ? examples/s]

Map:   0%|          | 0/52 [00:00<?, ? examples/s]

Map:   0%|          | 0/220 [00:00<?, ? examples/s]

The following columns in the training set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
c:\Users\danid\anaconda3\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 461
  Num Epochs = 5
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 75
  Number of trainable parameters = 134902275


  0%|          | 0/75 [00:00<?, ?it/s]

...

{'eval_loss': 0.758226752281189, 'eval_precision': {'precision': 0.7293650793650794}, 'eval_recall': {'recall': 0.7425328554360813}, 'eval_f1': {'f1': 0.7160370634354954}, 'eval_runtime': 2.3626, 'eval_samples_per_second': 22.009, 'eval_steps_per_second': 0.847, 'epoch': 5.0}


Model weights saved in ./results\checkpoint-75\pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from ./results\checkpoint-45 (score: 0.6782239079475403).
The following columns in the test set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 220
  Batch size = 32


{'train_runtime': 411.1427, 'train_samples_per_second': 5.606, 'train_steps_per_second': 0.182, 'train_loss': 0.5718584187825521, 'epoch': 5.0}


  0%|          | 0/7 [00:00<?, ?it/s]

Class: 0
Precision: 0.5348837209302325
Recall: 0.8214285714285714
F1-score: 0.647887323943662

Class: 1
Precision: 0.9
Recall: 0.7875
F1-score: 0.84

Class: 2
Precision: 0.5405405405405406
Recall: 0.625
F1-score: 0.5797101449275363

Class: macro avg
Precision: 0.658474753823591
Recall: 0.744642857142857
F1-score: 0.6891991562903993

Class: weighted avg
Precision: 0.8012456431061082
Recall: 0.7681818181818182
F1-score: 0.7776889532186532



In [ ]:
# Final evaluation
f1_against = report['1']['f1-score']
f1_favor = report['2']['f1-score']
tweeteval_result_atheism = (f1_against+f1_favor) / 2
print("Final F1 - Stance ATHEISM: " + str(tweeteval_result_atheism))

Final F1 - Stance ATHEISM: 0.7098550724637681


### Stance: feminist

In [ ]:
# Import dataset
train_dataset, val_dataset, test_dataset = raw_reader_and_tokenize("feminist")


# Create a Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

# Fine-tune the model on the training dataset
trainer.train()

# Generate predictions for the test set
test_predictions = trainer.predict(test_dataset).predictions
test_predictions = np.argmax(test_predictions, axis=-1)

# Compute the classification report
report = classification_report(test_dataset['label'], test_predictions, output_dict=True)

# Print the classification report
for label in report:
    if label != 'accuracy':
        print(f"Class: {label}")
        print(f"Precision: {report[label]['precision']}")
        print(f"Recall: {report[label]['recall']}")
        print(f"F1-score: {report[label]['f1-score']}\n")

Map:   0%|          | 0/597 [00:00<?, ? examples/s]

Map:   0%|          | 0/67 [00:00<?, ? examples/s]

Map:   0%|          | 0/285 [00:00<?, ? examples/s]

The following columns in the training set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
c:\Users\danid\anaconda3\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 597
  Num Epochs = 5
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 95
  Number of trainable parameters = 134902275


  0%|          | 0/95 [00:00<?, ?it/s]

...

{'eval_loss': 1.0821036100387573, 'eval_precision': {'precision': 0.5438228438228437}, 'eval_recall': {'recall': 0.5481185481185481}, 'eval_f1': {'f1': 0.5421310368118878}, 'eval_runtime': 4.415, 'eval_samples_per_second': 15.176, 'eval_steps_per_second': 0.68, 'epoch': 5.0}


Model weights saved in ./results\checkpoint-95\pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from ./results\checkpoint-19 (score: 0.8846456408500671).
The following columns in the test set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 285
  Batch size = 32


{'train_runtime': 735.8505, 'train_samples_per_second': 4.057, 'train_steps_per_second': 0.129, 'train_loss': 0.6242399115311472, 'epoch': 5.0}


  0%|          | 0/9 [00:00<?, ?it/s]

Class: 0
Precision: 0.49206349206349204
Recall: 0.7045454545454546
F1-score: 0.5794392523364486

Class: 1
Precision: 0.7156398104265402
Recall: 0.825136612021858
F1-score: 0.766497461928934

Class: 2
Precision: 0.36363636363636365
Recall: 0.06896551724137931
F1-score: 0.11594202898550723

Class: macro avg
Precision: 0.5237798887087987
Recall: 0.5328825279362306
F1-score: 0.48729291441696326

Class: weighted avg
Precision: 0.6094869756131915
Recall: 0.6526315789473685
F1-score: 0.6052245625156423



In [ ]:
# Final evaluation
f1_against = report['1']['f1-score']
f1_favor = report['2']['f1-score']
tweeteval_result_feminist = (f1_against+f1_favor) / 2
print("Final F1 - Stance FEMINIST: " + str(tweeteval_result_feminist))

Final F1 - Stance FEMINIST: 0.4412197454572206


### Stance: Abortion

In [ ]:
# Import dataset
train_dataset, val_dataset, test_dataset = raw_reader_and_tokenize("abortion")


# Create a Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

# Fine-tune the model on the training dataset
trainer.train()

# Generate predictions for the test set
test_predictions = trainer.predict(test_dataset).predictions
test_predictions = np.argmax(test_predictions, axis=-1)

# Compute the classification report
report = classification_report(test_dataset['label'], test_predictions, output_dict=True)

# Print the classification report
for label in report:
    if label != 'accuracy':
        print(f"Class: {label}")
        print(f"Precision: {report[label]['precision']}")
        print(f"Recall: {report[label]['recall']}")
        print(f"F1-score: {report[label]['f1-score']}\n")

Map:   0%|          | 0/587 [00:00<?, ? examples/s]

Map:   0%|          | 0/66 [00:00<?, ? examples/s]

Map:   0%|          | 0/280 [00:00<?, ? examples/s]

The following columns in the training set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
c:\Users\danid\anaconda3\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 587
  Num Epochs = 5
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 95
  Number of trainable parameters = 134902275


  0%|          | 0/95 [00:00<?, ?it/s]

...

{'eval_loss': 0.7481910586357117, 'eval_precision': {'precision': 0.7282765737874097}, 'eval_recall': {'recall': 0.7777777777777777}, 'eval_f1': {'f1': 0.7378578410836475}, 'eval_runtime': 3.1779, 'eval_samples_per_second': 20.768, 'eval_steps_per_second': 0.944, 'epoch': 5.0}


Model weights saved in ./results\checkpoint-95\pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from ./results\checkpoint-19 (score: 0.6615263223648071).
The following columns in the test set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 280
  Batch size = 32


{'train_runtime': 899.8938, 'train_samples_per_second': 3.261, 'train_steps_per_second': 0.106, 'train_loss': 0.4689614145379317, 'epoch': 5.0}


  0%|          | 0/9 [00:00<?, ?it/s]

Class: 0
Precision: 0.45454545454545453
Recall: 0.7777777777777778
F1-score: 0.5737704918032788

Class: 1
Precision: 0.7722222222222223
Recall: 0.7354497354497355
F1-score: 0.7533875338753387

Class: 2
Precision: 0.4782608695652174
Recall: 0.2391304347826087
F1-score: 0.3188405797101449

Class: macro avg
Precision: 0.5683428487776313
Recall: 0.584119316003374
F1-score: 0.5486662017962541

Class: weighted avg
Precision: 0.6728733766233768
Recall: 0.6607142857142857
F1-score: 0.6531306525009043



In [ ]:
# Final evaluation
f1_against = report['1']['f1-score']
f1_favor = report['2']['f1-score']
tweeteval_result_abortion = (f1_against+f1_favor) / 2
print("Final F1 - Stance ABORTION: " + str(tweeteval_result_abortion))

Final F1 - Stance ABORTION: 0.5361140567927418


### Stance: Hillary

In [ ]:
# Import dataset
train_dataset, val_dataset, test_dataset = raw_reader_and_tokenize("abortion")


# Create a Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

# Fine-tune the model on the training dataset
trainer.train()

# Generate predictions for the test set
test_predictions = trainer.predict(test_dataset).predictions
test_predictions = np.argmax(test_predictions, axis=-1)

# Compute the classification report
report = classification_report(test_dataset['label'], test_predictions, output_dict=True)

# Print the classification report
for label in report:
    if label != 'accuracy':
        print(f"Class: {label}")
        print(f"Precision: {report[label]['precision']}")
        print(f"Recall: {report[label]['recall']}")
        print(f"F1-score: {report[label]['f1-score']}\n")

Map:   0%|          | 0/587 [00:00<?, ? examples/s]

Map:   0%|          | 0/66 [00:00<?, ? examples/s]

Map:   0%|          | 0/280 [00:00<?, ? examples/s]

The following columns in the training set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
c:\Users\danid\anaconda3\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 587
  Num Epochs = 5
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 95
  Number of trainable parameters = 134902275


  0%|          | 0/95 [00:00<?, ?it/s]

...

{'eval_loss': 0.8488245606422424, 'eval_precision': {'precision': 0.734640522875817}, 'eval_recall': {'recall': 0.7592592592592594}, 'eval_f1': {'f1': 0.7378215654077723}, 'eval_runtime': 3.5002, 'eval_samples_per_second': 18.856, 'eval_steps_per_second': 0.857, 'epoch': 5.0}


Model weights saved in ./results\checkpoint-95\pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from ./results\checkpoint-57 (score: 0.7921849489212036).
The following columns in the test set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 280
  Batch size = 32


{'train_runtime': 520.1074, 'train_samples_per_second': 5.643, 'train_steps_per_second': 0.183, 'train_loss': 0.3170971117521587, 'epoch': 5.0}


  0%|          | 0/9 [00:00<?, ?it/s]

Class: 0
Precision: 0.5076923076923077
Recall: 0.7333333333333333
F1-score: 0.6

Class: 1
Precision: 0.8544303797468354
Recall: 0.7142857142857143
F1-score: 0.7780979827089337

Class: 2
Precision: 0.5263157894736842
Recall: 0.6521739130434783
F1-score: 0.5825242718446602

Class: macro avg
Precision: 0.6294794923042758
Recall: 0.6999309868875087
F1-score: 0.6535407515178646

Class: weighted avg
Precision: 0.7448000783360543
Recall: 0.7071428571428572
F1-score: 0.7173451258458674



In [ ]:
# Final evaluation
f1_against = report['1']['f1-score']
f1_favor = report['2']['f1-score']
tweeteval_result_hillary = (f1_against+f1_favor) / 2
print("Final F1 - Stance HILLARY: " + str(tweeteval_result_hillary))

Final F1 - Stance HILLARY: 0.680311127276797


The average F1 for all five datasets is:

In [ ]:
f1_all = [tweeteval_result, tweeteval_result_abortion, tweeteval_result_atheism, tweeteval_result_feminist, tweeteval_result_hillary]
f1_all = np.array(f1_all)
f1_all_dic = {"Climate":tweeteval_result, "Abortion": tweeteval_result_abortion, "Atheism": tweeteval_result_atheism, "Feminist": tweeteval_result_feminist, "Hillary": tweeteval_result_hillary}
print("Global F1 mean: " + str(np.mean(f1_all)))
print(f1_all_dic)

Global F1 mean: 0.5626566269041297
{'Climate': 0.44578313253012053, 'Abortion': 0.5361140567927418, 'Atheism': 0.7098550724637681, 'Feminist': 0.4412197454572206, 'Hillary': 0.680311127276797}


## How to improve the model: Hyperparameter tuning

### Stance: Climate

In [ ]:
train_dataset, val_dataset, test_dataset = raw_reader_and_tokenize("climate")

Map:   0%|          | 0/355 [00:00<?, ? examples/s]

Map:   0%|          | 0/40 [00:00<?, ? examples/s]

Map:   0%|          | 0/169 [00:00<?, ? examples/s]

In [ ]:
import optuna

# Define the objective function for Optuna
def objective(trial):
    # Define the hyperparameters to optimize
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-3, log=True)
    num_train_epochs = trial.suggest_int("num_train_epochs", 5, 10)
    per_device_train_batch_size = trial.suggest_categorical("per_device_train_batch_size", [32, 64])
    per_device_eval_batch_size = trial.suggest_categorical("per_device_eval_batch_size", [16, 32, 64])

    # Define the training arguments
    training_args = TrainingArguments(
        output_dir='./results',
        evaluation_strategy='epoch',
        save_strategy='epoch',
        learning_rate=learning_rate,
        per_device_train_batch_size=per_device_train_batch_size,
        per_device_eval_batch_size=per_device_eval_batch_size,
        num_train_epochs=num_train_epochs,
        logging_dir='./logs',
        load_best_model_at_end=True,
        seed=1
    )

    # Create a Trainer instance
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=compute_metrics
    )

    # Fine-tune the model on the training dataset
    trainer.train()

    # Evaluate the model on the validation dataset
    eval_result = trainer.evaluate(eval_dataset=val_dataset)

    return eval_result["eval_f1"]

# Create a study object and optimize the hyperparameters
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=10)


[I 2023-03-31 19:18:27,205] A new study created in memory with name: no-name-936b3d0b-3a1b-46aa-bc5f-066cbf0fa27f
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the training set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
c:\Users\danid\anaconda3\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running tr

  0%|          | 0/96 [00:00<?, ?it/s]

The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 40
  Batch size = 16


  0%|          | 0/3 [00:00<?, ?it/s]

Trainer is attempting to log a value of "{'precision': 0.6023550724637681}" of type <class 'dict'> for key "eval/precision" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'recall': 0.6115779645191409}" of type <class 'dict'> for key "eval/recall" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'f1': 0.606060606060606}" of type <class 'dict'> for key "eval/f1" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Saving model checkpoint to ./results\checkpoint-12
Configuration saved in ./results\checkpoint-12\config.json


...

{'eval_loss': 0.8558170199394226, 'eval_precision': {'precision': 0.17500000000000002}, 'eval_recall': {'recall': 0.3333333333333333}, 'eval_f1': {'f1': 0.2295081967213115}, 'eval_runtime': 2.0194, 'eval_samples_per_second': 19.808, 'eval_steps_per_second': 1.486, 'epoch': 7.0}


Model weights saved in ./results\checkpoint-84\pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from ./results\checkpoint-84 (score: 0.8558170199394226).
The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 40
  Batch size = 16


{'train_runtime': 498.8641, 'train_samples_per_second': 4.981, 'train_steps_per_second': 0.168, 'train_loss': 0.8337012699672154, 'epoch': 7.0}


  0%|          | 0/3 [00:00<?, ?it/s]

c:\Users\danid\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Trainer is attempting to log a value of "{'precision': 0.17500000000000002}" of type <class 'dict'> for key "eval/precision" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'recall': 0.3333333333333333}" of type <class 'dict'> for key "eval/recall" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'f1': 0.2295081967213115}" of type <class 'dict'> for key "eval/f1" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
[W 2023-03-31 20:3

We can now call the `best_params` argument and evaluate the model on the test set. Because of some kernel crashes, I'll manually input in the code below the parameters that yielded better results from this tuning. 

In [45]:
# Import dataset
train_dataset, val_dataset, test_dataset = raw_reader_and_tokenize("climate")

best_training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    save_strategy='epoch',
    learning_rate=1e-4,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=16,
    num_train_epochs=8,
    logging_dir='./logs',
    load_best_model_at_end=True,
    seed=1
)

# Create a Trainer instance
trainer = Trainer(
    model=model,
    args=best_training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

# Fine-tune the model on the training dataset
trainer.train()

# Generate predictions for the test set
test_predictions = trainer.predict(test_dataset).predictions
test_predictions = np.argmax(test_predictions, axis=-1)



Map:   0%|          | 0/355 [00:00<?, ? examples/s]

Map:   0%|          | 0/40 [00:00<?, ? examples/s]

Map:   0%|          | 0/169 [00:00<?, ? examples/s]

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,No log,0.492835,{'precision': 0.5498721227621483},{'recall': 0.5760971055088703},{'f1': 0.5623885918003565}
2,No log,0.446478,{'precision': 0.5488721804511277},{'recall': 0.5798319327731093},{'f1': 0.5634920634920634}
3,No log,0.339875,{'precision': 0.5993265993265994},{'recall': 0.6311858076563959},{'f1': 0.6148394241417497}
4,No log,0.493289,{'precision': 0.5488721804511277},{'recall': 0.5798319327731093},{'f1': 0.5634920634920634}
5,No log,0.694449,{'precision': 0.566951566951567},{'recall': 0.5686274509803922},{'f1': 0.5583333333333335}
6,No log,0.458407,{'precision': 0.6041666666666666},{'recall': 0.6274509803921569},{'f1': 0.6141414141414141}
7,No log,0.473179,{'precision': 0.6041666666666666},{'recall': 0.6274509803921569},{'f1': 0.6141414141414141}
8,No log,0.476864,{'precision': 0.6041666666666666},{'recall': 0.6274509803921569},{'f1': 0.6141414141414141}


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Trainer is attempting to log a value of "{'precision': 0.5498721227621483}" of type <class 'dict'> for key "eval/precision" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'recall': 0.5760971055088703}" of type <class 'dict'> for key "eval/recall" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'f1': 0.5623885918003565}" of type <class 'dict'> for key "eval/f1" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
/usr/local/lib/python3.

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


NameError: ignored

In [49]:
# Compute the classification report
report = classification_report(test_dataset['label'], test_predictions, output_dict=True)

# Print the classification report
for label in report:
    if label != 'accuracy':
        print(f"Class: {label}")
        print(f"Precision: {report[label]['precision']}")
        print(f"Recall: {report[label]['recall']}")
        print(f"F1-score: {report[label]['f1-score']}\n")

Class: 0
Precision: 0.6595744680851063
Recall: 0.8857142857142857
F1-score: 0.7560975609756098

Class: 1
Precision: 0.0
Recall: 0.0
F1-score: 0.0

Class: 2
Precision: 0.8934426229508197
Recall: 0.8861788617886179
F1-score: 0.8897959183673471

Class: macro avg
Precision: 0.517672363678642
Recall: 0.5906310491676345
F1-score: 0.5486311597809856

Class: weighted avg
Precision: 0.7868553195617133
Recall: 0.8284023668639053
F1-score: 0.8041911987771008



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Even after hyperparameter tuning, the results do not improve substantially for the climate dataset.

## Where is our model failing?



We can look at some examples of where the model fails. Let's take, again, the climate dataset.

In [63]:
climate_outcomes = pd.DataFrame(test_dataset["text"])
climate_outcomes["label_real"]= test_dataset["label"]
climate_outcomes["label_predicted"]= test_predictions
climate_outcomes["wrong_prediction"]= climate_outcomes["label_predicted"]!=climate_outcomes["label_real"]
wrong_outcomes = climate_outcomes[climate_outcomes["wrong_prediction"]==True]


In [84]:
pd.set_option('max_colwidth', 200)

wrong_outcomes

,0,label_real,label_predicted,wrong_prediction
12,grow food not lawns Lawns are Ecocide in your yard. #Patriarchy is Ecocide of your own species #DominoEffect #SemST,2,0,True
15,Citizens have a right to access information that would protect their lives against #risk #CLIMWARN #CFCC15 #vulnerability #SemST,2,0,True
23,It's raining and feels like fall I don't know what kinda summer this is #isReal #SemST,2,0,True
29,As Western Canada Burns Harper's got his head so deep in the #tarsands he can see #ChristyClark's ankles #BCwildfire #cdnpoli #SemST,0,2,True
43,Turn the Liverpool Plains into a coal mine...is there any bad idea this government doesn't like? #FoodBowl #Shenhua #auspol #SemST,2,0,True
46,The Weather app keeps taunting us with rain. #PNW #drought #SemST,0,2,True
49,Haven't moved an inch since yesterday. #bcpoli #vancouver #bcwildfire #yvr #realrenewables #SemST,0,2,True
65,@user tackles difficult challenges of growth needs and unemployment alongside #CFCC15. #SemST,2,0,True
85,@user @user @user @user @user Criminal capitalist wrecked economy08Now misleadingInvestors as to Risk of #Ecocide #SemST,2,0,True
87,@user @user @user @user #CSOTA #fossilfuels corp misleading investors about liability for #Ecocide #SemST,2,0,True


We see some examples of misclassification as 'against' (2) where the correct label is 'in favor' (1), which seem reasonable. For instance, in line 136 the bigram 'climate deniers' is probably helping the classifier lean towards 'in favor', as it is a typical way to critize those against climate change narrative. However, the intention of the tweet is to critize the use of the term, which confuses the algorithm.

Other cases are less clear: it would be reasonable to have them correctly classied - such is the case of line 147, where the hashtag #ClimateFreud is used.

As we can see in the table below, the main misclasifications happen following this logic:
Neutral tweets classified as 'against'
'Against' tweets classified as neutral
'Against tweets classified as 'In favor'.



In [90]:
pd.crosstab(wrong_outcomes['label_real'], wrong_outcomes['label_predicted'], normalize='index')

label_predicted,0,2
label_real,,
0,0.000000,1.000000
1,0.181818,0.818182
2,1.000000,0.000000
